In [ ]:
import utils
import re

import                                  torch
import                                  pyaudio
import                                  wave
import                                  numpy                                               as  np
from    transformers            import  WhisperProcessor, WhisperForConditionalGeneration
from    scipy.io                import  wavfile
from    transformers            import  pipeline
# Parameters for audio input
FORMAT = pyaudio.paInt16   # Format of audio samples
CHANNELS = 1               # Number of audio channels (1 for mono, 2 for stereo)
RATE = 16000               # Sampling rate (samples per second)
CHUNK = 1024               # Number of frames per buffer


# 1.Audio Extact

if tou want to use the bash command

```ffmpeg -i .input_file.mp4 -vn -acodec pcm_s16le -ar 16000 -ac 1  ./output.wav```

# 2.Audio-to-text

models description :

https://huggingface.co/openai/whisper-large-v2

https://huggingface.co/jonatasgrosman/wav2vec2-large-xlsr-53-english



In [ ]:
index = -1
modelss = [
            # "openai/whisper-tiny",   #39m params useless
            "openai/whisper-base",   #74m params
            "openai/whisper-small",  #244m params very well
            "openai/whisper-medium", #769m params
           ]
# load model and processor
processor = WhisperProcessor.from_pretrained(modelss[index])
model = WhisperForConditionalGeneration.from_pretrained(modelss[index]).to('cuda')
model.config.forced_decoder_ids = None


In [ ]:
samplerat1, data1 = wavfile.read('output.wav')
data1= data1/10

When utilizing the output from FFMPEG after loading 'output.wav', it's important to note that the amplitude should be divided by 10. Additionally, the Speech-to-Text (STT) system has a memory of 27 seconds.

In [ ]:
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="transcribe")
text = ''

S_rate = 16000
for index in range(len(data1)//(25*S_rate)):
    input_features = processor(data1[(S_rate*index*25):(S_rate*(index+1)*25)], sampling_rate=S_rate, return_tensors="pt").input_features.to('cuda')
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    text += transcription[0] + ' '


In [ ]:
text

In [ ]:
total_lenght = len(text)
' '.join(text.split(' ')[0:(total_lenght//3)]) 

In [ ]:

# Define the regular expression pattern
paragraph_pattern = r'\.'

# Your long transcript
transcript = text

# Split the transcript into paragraphs
paragraphs = re.split(paragraph_pattern, transcript)

# Optionally, strip leading/trailing whitespace
paragraphs = [p.strip() for p in paragraphs]


# Using a list comprehension to remove empty strings
filtered_list = [item for item in paragraphs if item.strip() != ""]

print(filtered_list)


# 3.Summurazation

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
print(summarizer(' '.join(filtered_list[0:(len(filtered_list)//3)]) , min_length=90, do_sample=True))
